# HW10

## Q1.

Implement a Key-Value Search true, which allows no duplicates, but rather, updates the value associated with the key. This will change how `insert` works. Inherit from the augmented tree:

`class KeyValueBinarySearchTree(AugmentedBinarySearchTree):`

- The constructor should look like this: `def __init__(self, key_value_tuple, parent=None):`. Pick the key and value out separately in the constructor, and initialize the super with just the key, setting an instance variable `self.value` to the value.
- insert wont duplicate any more, and `addLeftChild` and `addRightChild` will need to take the tuple in as they call the constructor for us.
- implement a `__getitem__`, `__setitem__`, and `__delitem__` so that you can use code like:

`btreekv['f']` for searching

`btreekv['f']=10` for inserting

In [56]:
#import base class
from BinaryTrees import AugmentedBinarySearchTree

class KeyValueBinarySearchTree(AugmentedBinarySearchTree):
    """
    A key-value binary search tree
    
    NOTE
    ----
    No duplicates allowed, insert k-v pair with existing key will update the corresponding value
    
    EXAMPLE
    -------
    >>> btreekv = KeyValueBinarySearchTree(('a', 1))
    >>> btreekv['b'] = 2
    >>> btreekv['c'] = 3
    >>> btreekv['a']
    1
    >>> btreekv['b']
    2
    >>> btreekv['a'] = 4
    >>> btreekv['a']
    4
    >>> del btreekv['a']
    >>> [(e.data, e.value) for e in list(btreekv)]
    [('b', 2), ('c', 3)]
    
    """
    
    def __init__(self, kv_tuple, parent=None):
        key = kv_tuple[0]
        val = kv_tuple[1]
        super().__init__(key, parent)
        self.value = val
    
    # add k-v tuple
    def addLeftChild(self, kv_tuple): 
        n = self.__class__(kv_tuple, self)
        self.left = n
        return n
        
    def addRightChild(self, kv_tuple):
        n = self.__class__(kv_tuple, self)
        self.right = n
        return n
    
    def insert(self, kv_tuple):
        data = kv_tuple[0] # data is the key
        if data < self.data:
            if self.hasLeftChild():
                self.left.insert(kv_tuple)
            else:
                self.addLeftChild(kv_tuple)
                self._insert_hook()
        elif data > self.data:
            if self.hasRightChild():
                self.right.insert(kv_tuple)
            else:
                self.addRightChild(kv_tuple)
                self._insert_hook()
        else: #same key found
            self.value = kv_tuple[1] # update the value
            self._insert_hook()
            
    
    def _remove(self, node):
        """
        Override the _remove function for delete function in parent class
        Copy data (key) as well as value from successor to the node to be deleted
        
        Take special care for the case when deleting the root with only one child
        Since we cannot change self (immutable), we cannot set self equals to its child in order to delete the root
        Instead we can copy the data of the root's child into root and delete the child of root 
        And reparent the children of the root's child
        """
            if node.isLeaf():
            if node.isLeftChild():
                node.parent.left = None
            else:
                node.parent.right = None
            node._remove_hook(by=node.count)
            del node
        elif node.hasBothChildren():
            s = node.successor()
            s.spliceOut()
            s._remove_hook(by=s.count)
            node.data = s.data
            node.value = s.value # replace value as well
            node._remove_hook(up=True, by = s.count - node.count)
            node.count = s.count
            del s #the node became the successor
        else: # one child case
            if node.hasLeftChild():
                if node.isLeftChild():
                    node.left.parent = node.parent
                    node.parent.left = node.left
                elif node.isRightChild():
                    node.left.parent = node.parent
                    node.parent.right = node.left
                else: # node is root
                    node.data = node.left.data
                    node.value = node.left.value
                    left_node = node.left
                    nll = left_node.left
                    nlr = left_node.right
                    if nll:
                        nll.parent = node
                    node.left = nll
                    if nlr:
                        nrl.parent = node
                    node.right = nrl
                    del left_node
                node._remove_hook(by=node.count)
                del node
            else:
                if node.isLeftChild():
                    node.right.parent = node.parent
                    node.parent.left = node.right
                elif node.isRightChild():
                    node.right.parent = node.parent
                    node.parent.right = node.right
                else: # node is root
                    node.data = node.right.data
                    node.value = node.right.value
                    right_node = node.right
                    nrl = right_node.left
                    nrr = right_node.right
                    if nrl:
                        nrl.parent = node
                    node.left = nrl
                    if nrr:
                        nrr.parent = node
                    node.right = nrr
                    del right_node
                node._remove_hook(by=node.count)
                del node
            
    def __getitem__(self, key):
        node = self.search(key)
        if node is not None:
            return node.value
        else:
            return None

    def __setitem__(self, key, val):
        self.insert((key, val))
        
    def __delitem__(self, key):
        self.delete(key)
        

In [57]:
from doctest import run_docstring_examples as dtest
dtest(KeyValueBinarySearchTree, globals(), verbose=True)

Finding tests in NoName
Trying:
    btreekv = KeyValueBinarySearchTree(('a', 1))
Expecting nothing
ok
Trying:
    btreekv['b'] = 2
Expecting nothing
ok
Trying:
    btreekv['c'] = 3
Expecting nothing
ok
Trying:
    btreekv['a']
Expecting:
    1
ok
Trying:
    btreekv['b']
Expecting:
    2
ok
Trying:
    btreekv['a'] = 4
Expecting nothing
ok
Trying:
    btreekv['a']
Expecting:
    4
ok
Trying:
    del btreekv['a']
Expecting nothing
ok
Trying:
    [(e.data, e.value) for e in list(btreekv)]
Expecting:
    [('b', 2), ('c', 3)]
ok


It should all make sense....

In [46]:
btreekv = KeyValueBinarySearchTree(('f', 3))

In [47]:
kvdata=zip(list('jeihrifhkdfks'), range(13))

In [48]:
for k,v in kvdata:
    btreekv[k]=v

In [49]:
[(e.data, e.value) for e in list(btreekv)]

[('d', 9),
 ('e', 1),
 ('f', 10),
 ('h', 7),
 ('i', 5),
 ('j', 0),
 ('k', 11),
 ('r', 4),
 ('s', 12)]

In [50]:
del btreekv['f']

In [51]:
[(e.data, e.value) for e in list(btreekv)]

[('d', 9),
 ('e', 1),
 ('h', 7),
 ('i', 5),
 ('j', 0),
 ('k', 11),
 ('r', 4),
 ('s', 12)]